In [ ]:
# 导入相关包
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
#这是巴西某4S汽车连锁店的实际销售数据，大小约3.5G，包含了从2017年3月31日到2020年4月1日大约3000万行的销售数据
# 本案例数据下载百度网盘地址：

In [ ]:
# 读取数据，设置分割符号
df = pd.read_csv("d:/temp/Sales Report.csv", iterator=True, sep=';')
# data = df.get_chunk(3000000)
data = df.get_chunk(30000000)

In [ ]:
del data


In [ ]:
data = data.sample(n=3000000)
data.to_csv("d:/temp/Jupyter/baxishuju.csv")
# tp = pd.read_csv('Sales Report.csv', sep='\t', iterator=True, chunksize=1000)
# df = pd.concat(tp, ignore_index=True)

In [ ]:
data=pd.read_csv("d:/temp/Jupyter/baxishuju.csv")
data.info()

In [ ]:
data.sample(2)

In [ ]:
data.dtypes

In [ ]:
# 查看NULL的数据：
data.isnull().sum()

In [ ]:
data.notnull().sum()

In [ ]:
# 查看数据的标准差，最大，最下值
data.describe()

In [ ]:
#这里的数据量比较多，数据相对比较大，这里很明显可以看出的Product Cost这里有个负数
data[data['Product Cost'] < 0]

In [ ]:
len(data[data['Product Cost'] < 0])

In [ ]:
#每销售出一个该产品的成本，这里如果为负数，直接删除处理
data.drop(index=data[data['Product Cost'] < 0].index, inplace=True)

In [ ]:
# 数据清洗，这里有48381条数据是重复的，删除这些数据
data[data.duplicated()]

In [ ]:
# 删除重复的数据
# 这里的重复的数据是完全重复的，所有的值都是相同的，
# 这里只能判断为异常数据，直接删除掉
data.drop(index=data[data.duplicated()].index,inplace=True)

In [ ]:
data['Sale Date Time'] = pd.to_datetime(data['Sale Date Time'])
data.info()

In [ ]:
data['Sale Date Time'].unique()

In [ ]:
data['year'] = data['Sale Date Time'].dt.year
data['month'] = data['Sale Date Time'].dt.month
data['day'] = data['Sale Date Time'].dt.day
data['hour'] = data['Sale Date Time'].dt.hour
data['day of week'] = data['Sale Date Time'].dt.day_name()
data.head()

In [ ]:
data.to_csv("./baxidata.csv")

In [ ]:
# 年销售额情况
data.groupby('year')['Order Number'].count()

In [ ]:
data.groupby(['year'])['Order Number'].count().plot()

In [ ]:
data.groupby(['year','month'])['Order Number'].count()

In [ ]:
data.groupby(['year','month'])['Order Number'].count().plot()

In [ ]:
data.groupby(['year','month'])['Order Number'].count().idxmax()

In [ ]:
周的销售情况

In [ ]:
'day of week'  
data.groupby(['day of week'],sort=False)['Order Number'].count()

In [ ]:
data.groupby(['day of week'],sort=False)['Order Number'].count().plot()

In [ ]:
data.groupby(['year','month','day of week'])['Order Number'].count()

In [ ]:
data.groupby(['year','month','day of week'])['Order Number'].count().plot()

In [ ]:
# 日的销售情况
data.groupby(['day'])['Order Number'].count()

In [ ]:
data.groupby(['hour'])['Order Number'].count()

In [ ]:
data.groupby(['hour'])['Order Number'].count().plot()

In [ ]:
##销售预测按日
# 将销售时间设置成索引
data.set_index('Sale Date Time', inplace=True, drop=True)
# 将数据重新整理成以天来统计每天销售额的汇总
day_data = data.resample('D')['Total'].sum()
day_data

In [ ]:
# 挑选2018年1月1日到2020年2月29的数据
train_day_data = day_data[day_data.index >= '2018-01-01']
train_day_data = train_day_data[train_day_data.index <= '2019-12-31']

train_day_data.to_excel('./日销售数据.xlsx')

In [ ]:

data = pd.read_excel('./日销售数据.xlsx',engine="openpyxl")

data.rename(columns={'Sale Date Time': 'date1'}, inplace=True)
data

In [ ]:

data['Total'] = round(data['Total'] / 100000, 4)

复购率
总体复购率：
这里是按这份数据所在的时间段，计算购买次数大于2次的用户，再除于总的用户数，这里得排除的一个数据是用户Client这里，有一个数据是Customer not informed（客户没有提供名字的情况），这条数据得排除了，所以计算购买次数大于2的用户和总用户数对应减去1

In [ ]:


import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = 'SimHei'

plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

plt.figure(figsize=(18, 8), dpi=256)

data['Total'][:-30].plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(data['Total'][:-30])
plt.figure(figsize=(18, 8), dpi=256)

In [ ]:

from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(data['Total'][:-30])
plt.figure(figsize=(18, 8), dpi=256)

In [ ]:

from statsmodels.tsa.stattools import adfuller as ADF 
print(ADF(data['Total'][:-30]))

In [ ]:
D_data = data['Total'][:-30].diff().dropna()
print('一阶段差分检验结果：', ADF(D_data))

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
print('白噪声检验结果：', acorr_ljungbox(D_data, lags=1,return_df=True))

In [ ]:

from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
from itertools import product
ps = range(0, 5)
qs = range(0, 5)
parameters = product(ps, qs)
parameters_list = list(parameters)

best_aic = float('inf')
results = []
for param in parameters_list:
    try:
        model = ARIMA(data['Total'][:-30], order=(param[0], 1, param[1])).fit()
    except ValueError:
        print("参数错误：", param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = model.aic
        best_param = param
    results.append([param, model.aic])
results_table = pd.DataFrame(results)
results_table.columns = ['parameters', 'aic']
print("最优模型", best_model.summary())

In [ ]:
best_model.forecast(30)

In [ ]:
from sklearn.metrics import mean_absolute_error
pred_y = best_model.forecast(30)
test_y = data['Total'][-30:].values
mean_absolute_error(test_y, pred_y)

In [ ]:
plt.figure(figsize=(14, 7), dpi=256)
plt.plot(data['date1'][-30:], test_y, label='实际')
plt.plot(data['date1'][-30:], pred_y, label='预测')
plt.xticks(data['date1'][-30:], rotation=70)
plt.legend(loc=3)

In [ ]:
data=pd.read_csv("./baxidata.csv")
# 计算每个客户的购买次数，这里使用了nunique(),统计不同订单号的个数
client_data = data.groupby('Client').nunique()['Order Number']
# 重命名列
client_data = client_data.reset_index().rename(columns={'Order Number': 
                                                        'user_num'})
client_data.sort_values('user_num', ascending=False)         

In [ ]:
# 总复购率
print('总复购率:',
round((len(client_data[client_data['user_num'] > 1])-1) / (len(client_data) - 1),4) * 100, "%")

一个月内复购率
这里的一个月内复购率的定义是：从月初的1号到月底这段时间内，用户复购的比率。

In [ ]:
data['Sale Date Time'] = pd.to_datetime(data['Sale Date Time'])
data.info()
data.to_csv("./baxidata.csv")

In [ ]:
# 这里的销售时间是datetime格式，增加个辅助列，转换成2017-01这样的年月显示
def parse_year_month(x):
    if x.month >= 10:
        return str(x.year) + "-" + str(x.month)
    else:
        return str(x.year) + "-0" + str(x.month)
data['year_month'] = data['Sale Date Time'].apply(parse_year_month)
# 统计每个月用户的购买次数

y_m_data = data.groupby(['year_month','Client']).nunique()['Order Number'].reset_index()
y_m_data

In [ ]:
# 每个月的复购率
# 保存临时数据，用于构建每月的复购率的DataFrame
month_list = []
rate_list = []
# 循环计算每个月的复购率，这里直接遍历每个月
for every_m in y_m_data['year_month'].unique():
#     获取每个月用户的购买次数的数据
    temp = y_m_data[y_m_data['year_month'] == every_m]

    month_list.append(every_m)
#     选出购买次数>1的数据，获取数据的长度（用户数）- 1 再除以当月的总用户数 -1 
    rate_list.append(round((len(temp[temp['Order Number'] > 1])-1) / (len(temp) -1), 4))
#将数据转换成DataFrame
t_1 = {'month': month_list, 'rate': rate_list}
rate_data = pd.DataFrame(t_1)
rate_data

In [ ]:
y_m_data['year_month'].unique()

In [ ]:
temp['Order Number']

购买次数的情况

In [ ]:
# 这里只挑选了2019年9月这个月的购买次数来做分析，其他月份的可以类推。

data_201909 = y_m_data[y_m_data['year_month'] == '2019-09'].copy()
#重命名Order Number为购买次数buy_frequency
data_201909.rename(columns={'Order Number': 'buy_frequency'}, inplace=True)
print(data_201909.min())
print(data_201909.max())
data_201909.head()

In [ ]:
# 设置数据的区间
bins = [0, 1, 2, 5, 10, 50, 100, 100000]
per_frequency = pd.cut(data_201909['buy_frequency'], bins)
print(per_frequency.value_counts())
per_frequency.value_counts().plot(kind='bar')

RFM模型分析用户的价值

In [ ]:
# 这里只针对2019年9月份的用户价值进行分类，其他可以类推。
# 按月份提取每个月用户的R、F、M值
RFM_data_all = data.groupby(['year_month','Client']).agg({'Order Number': "nunique",'Sale Date Time': max,'Total': sum})
RFM_data_all.reset_index(inplace=True)

In [ ]:
# 保存一份数据，下次直接读取该数据集就可以，省时间
RFM_data_all.to_excel('RFM_data_all.xlsx', index=False)

In [ ]:
# 提取2018年4月份的数据
RFM_data_201909  = RFM_data_all[RFM_data_all['year_month'] == '2019-09']
RFM_data_201909                 

In [ ]:
# 参考时间，这里随便设置里2019-11-30 23:59:59，不让R值为0，这里的R以天作为单位
import datetime
reference_time = datetime.datetime.strptime('2019-11-30 23:59:59',"%Y-%m-%d %H:%M:%S")
# 构建R指标
RFM_data_201909['R'] = RFM_data_201909['Sale Date Time'].apply(lambda x: (reference_time - x).days)
# 重新命名列
RFM_data_201909.rename(columns={'Order Number': 'F', 'Total': 'M'}, inplace=True)
# 排序查看异常值
RFM_data_201909.sort_values('M', ascending=False)

In [ ]:
# 这里有个异常值，标记为客户没有提及姓名的，直接删除处理。
RFM_data_201909.drop(index=35727, inplace=True)

In [ ]:
RFM_data = RFM_data_201909[['R', 'F', 'M']]

In [ ]:
RFM_data

In [ ]:
df1=RFM_data.apply(lambda x:[x.min(),x.max(),x.mean()])
df1.rename(index={0: "min", 1: "max", 2: "mean"})

In [ ]:
# def kmeans(X, n_clusters):
#     ss = StandardScaler()
#     X = ss.fit_transform(X)
#     km = KMeans(n_clusters=n_clusters)
#     km.fit(X)
#     y_pred = km.predict(X)
#     plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
#     plt.title("K-means")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
#数据规范化
ss  = StandardScaler()
train = ss.fit_transform(RFM_data)
# 模型进行训练，这里直接聚类成5类。
kmeans_model = KMeans(n_clusters=5)
kmeans_model.fit(train)
#查看聚类中心
cluster = pd.DataFrame(kmeans_model.cluster_centers_, columns=['R', 'F', 'M'])
cluster

分群0：R小，F小，M小，这类属于一般价值客户。

分群1，R大，F小，M小，这类属于一般发展客户。

分群2，R小，F大，M大，这类属于重点保持客户。

分群3、4 这类，R小，F、M大，这类都属于高价值客户。

In [ ]:
# 将分群的结果合并到RFM_data数据里看下原数据。
RFM_data['sk5_label'] = kmeans_model.labels_

In [ ]:
RFM_data

In [ ]:
RFM_data.loc[RFM_data["R"] < RFM_data["R"].median(), "Rcut"] = 0
RFM_data.loc[RFM_data["R"] >= RFM_data["R"].median(), "Rcut"] = 1

In [ ]:
RFM_data["Mcut"]= np.where(RFM_data["M"]<RFM_data["M"].median(),0,1)
RFM_data["Mcut"]= np.where(RFM_data["M"]>=RFM_data["M"].median(),0,1)

In [ ]:
RFM_data.sample(3)

In [ ]:
RFM_data.describe()

In [ ]:
RFM_data[RFM_data['sk5_label'] == 2]

In [ ]:
RFM_data[RFM_data['sk5_label'] == 4]

In [ ]:
RFM_data[RFM_data['sk5_label'] == 3]

In [ ]:
RFM_data[RFM_data['sk5_label'] == 1]

In [ ]:
RFM_data.to_csv("./RFM_data.csv")

#用户月留存率
这里统计用户月存留率是上个月与当前月都有购买的用户的数量/(除以)上个月的总用户数（去重）。类似流失率，这里不同的是都是上月流向下月的，不是1->2->3这样的流向，而是1->2， 2->3这样的流向。

In [ ]:
# 提取每个月的用户（去重）
every_month_user = data.groupby(['year_month', 'Client'])['Order Number'].nunique().reset_index()
every_month_user

In [ ]:
# 获取每个月份的列表，循环遍历计算上个月与当前月的留存率
year_month = every_month_user['year_month'].unique()
# 保存月留存率的列表
list_month_rate = []
for i, month in enumerate(year_month):
#     计算上个月与当前月的留存率
    if i>= 1:
#         获取当前月的用户（上面groupby已去重）
        this_month_client = every_month_user[every_month_user['year_month'] == month]['Client']
#        获取上个月的用户（上面groupby已去重）
        previous_month_client = every_month_user[every_month_user['year_month'] == year_month[i-1]]['Client']
#         计算留存率，这里用的是上个月与当前月用户的交集个数/上个月的用户数（去重）
        rate = round(len (set(this_month_client) & set(previous_month_client))/ len(previous_month_client),2)
#         用列表保存数据，并构建DataFrame用户绘图
        b = [month, rate]
        list_month_rate.append(b)
# 构建DataFrame
rate_data = pd.DataFrame(list_month_rate, columns=['year_month', 'rate'])
rate_data

In [ ]:
# 产品角度的分析，使用聚类分析进行产品的分类
# 这里先获取每个月的产品的成本C，订单量F，销售总金额M，这里只挑选了2019年9月一个月的产品数据来分析

month_product_data = data.groupby(['year_month', 'Product']).agg({'Product Cost': 'mean','Order Number': 'nunique','Total': 'sum'}).reset_index()

# 重命名
month_product_data.rename(columns={'Product Cost': 'C', 'Order Number': 'F', 'Total': 'M'}, inplace=True)
month_product_data.to_excel('./month_product_data.xlsx', index=False)                                  
# 选择2019年9月的数据
# 这里只取一个月的产品进行聚类
month_product_201909 = month_product_data[month_product_data['year_month'] == '2019-09']
month_product_201909

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
ss = StandardScaler()
# 获取需要的数据
X = month_product_201909[['C', 'F', 'M']]
# 数据规范化
train_X = ss.fit_transform(X)
# 设置聚类数4个
dbscan_model = DBSCAN(min_samples=10)
# 模型训练
dbscan_model.fit(train_X)
# 将聚类的结果合并到原数据集上。
month_product_201909['labels'] = dbscan_model.labels_
# 查看聚类的分布情况
month_product_201909['labels'].value_counts()

In [ ]:
def dbscan(X, eps, min_samples):
    ss = StandardScaler()
    X = ss.fit_transform(X)
    db = DBSCAN(eps=eps, min_samples=min_samples)
    db.fit(X)
    y_pred = db.fit_predict(X)
    plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    plt.title("DBSCAN")

In [ ]:
dbscan(train_X,eps=0.4,min_samples=10)

In [ ]:
month_product_201909

In [ ]:
month_product_201909.to_csv("./month_product_201909.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
# 获取需要的数据
X = month_product_201909[['C', 'F', 'M']]
# 数据规范化
train_X = ss.fit_transform(X)
# 设置聚类数4个
kmeans_model2 = KMeans(n_clusters=5)
# 模型训练
kmeans_model2.fit(train_X)
# 将聚类的结果合并到原数据集上。
month_product_201909['labels_kmeans'] = kmeans_model2.labels_
# 查看聚类的分布情况
month_product_201909['labels_kmeans'].value_counts()

In [ ]:
def kmeans(X, n_clusters):
    ss = StandardScaler()
    X = ss.fit_transform(X)
    km = KMeans(n_clusters=n_clusters)
    km.fit(X)
    y_pred = km.predict(X)
    plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    plt.title("K-means")

In [ ]:
kmeans(X,n_clusters=5)

In [ ]:
def hier(X, n_clusters):
    from sklearn.cluster import AgglomerativeClustering
    ss = StandardScaler()
    X = ss.fit_transform(X)
    hier =AgglomerativeClustering(n_clusters=n_clusters)
    y_pred = hier.fit_predict(X)
    plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    plt.title("Hierarchical")

In [ ]:
hier(X,n_clusters=5)

In [ ]:
month_product_201909.to_csv("./month_product_201909.csv")